<a href="https://colab.research.google.com/github/wlgnsthd/BigdataAnalysis/blob/main/Naver_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pdb

client_id = "n1fBgJg4y7_vRANLV9n5" 
client_secret = "Mb_dBHAdsb"

#데이터 추출

In [ ]:
def get_request_url(url):
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
        if response.getcode() == 400:
            return None
    except:
        return None

In [ ]:
def getNaverSearchResult(sNode, search_text, page_start, display):
    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(search_text), page_start, display)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [ ]:
def getPostData(post, jsonResult):
    
    title = post['title']
    description = post['description']
    link = post['link']
    
    jsonResult.append({'title':title, 'link': link, "description": description})
    return

In [ ]:
jsonResult = []

# 'news', 'blog', 'cafearticle', 'encyc'
sNode = '' #검색할 항목 입력
search_text = '' #검색할 대상 입력
display_count = 100

jsonSearch = getNaverSearchResult(sNode, search_text, 1, display_count)

while ((jsonSearch != None) and (jsonSearch['display'] != 0)):
    for post in jsonSearch['items']:
        getPostData(post, jsonResult)
    nStart = jsonSearch['start'] + jsonSearch['display']
    jsonSearch = getNaverSearchResult(sNode, search_text, nStart, display_count)

with open('%s_naver_%s.json' % (search_text, sNode), 'w', encoding='utf-8-sig') as outfile:
    retJson = json.dumps(jsonResult,
                    indent=4, sort_keys=True,
                    ensure_ascii=False)
    outfile.write(retJson) 

#데이터정렬